# Download and extract from URL

In [6]:
from kfp.components import create_component_from_func, OutputPath
from typing import NamedTuple

%load_ext lab_black

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"


def download_and_extract_from_url(
    url: str,
    data_path: OutputPath(str),
    file_name: str = "data.zip",
):
    """
    Downloads and extractions packaged files (zip/tar/tar.gz) from a URL to a persistent volume.

            Parameters:
                    url: Url from which data has to be downloaded.
                    file_name: Name of the file to be stored.
            Returns:
                    data_path: Name of the folder that data is extracted to.
    """

    import logging
    import os
    from subprocess import run
    import sys
    import tarfile
    import zipfile

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )

    logging.info(f"Downloading from URL: {url}")
    logging.info(f"Downloading to persistent volume: {data_path}/{file_name}...")
    run(["wget", url, "-O", file_name], capture_output=True)
    logging.info("Downloaded finished.")

    def extract_zip():
        with zipfile.ZipFile(file_name, "r") as zip_ref:
            zip_ref.extractall(data_path)

    def is_within_directory(directory, target):
        abs_directory = os.path.abspath(directory)
        abs_target = os.path.abspath(target)
        prefix = os.path.commonprefix([abs_directory, abs_target])
        return prefix == abs_directory

    def safe_extract(tar, path=".", members=None, *, numeric_owner=False):
        for member in tar.getmembers():
            member_path = os.path.join(path, member.name)
            if not is_within_directory(path, member_path):
                raise Exception("Attempted Path Traversal in Tar File")
        tar.extractall(path, members, numeric_owner=numeric_owner)

    def extract_tar_gz():
        with tarfile.open(file_name, "r:gz") as tar_gz_ref:
            safe_extract(tar_gz_ref, data_path)

    def extract_tar():
        with tarfile.open(file_name, "r:") as tar_ref:
            safe_extract(tar_ref, args.dataPath)

    if file_name.endswith(("zip", "tar.gz", "tar")):
        logging.info(f"Detected archive file. Extracting to {data_path}...")
        if file_name.endswith("zip"):
            extract_zip()
        elif file_name.endswith("tar.gz"):
            extract_tar_gz()
        elif file_name.endswith("tar"):
            extract_tar()
        logging.info(f"Removing {file_name} again...")
        if os.path.isfile(file_name):
            os.remove(file_name)
        else:
            logging.error(f"Error: file {file_name} not found!")

    logging.info(f"Contents of {data_path}:")
    logging.info(os.listdir(data_path))
    logging.info("Finished.")


download_and_extract_from_url_comp = create_component_from_func(
    func=download_and_extract_from_url,
    output_component_file="component.yaml",
    base_image=BASE_IMAGE,
)